# ANSI Migration Guide - Pandas API on Spark
ANSI mode is now on by default for Pandas API on Spark. This guide helps you understand the key behavior differences you’ll see.
In short, with ANSI mode on, Pandas API on Spark behavior matches native pandas in cases where Pandas API on Spark with ANSI off did not.

## Behavior Change
### String Number Comparison
**ANSI off:** Spark implicitly casts numbers and strings, so `1` and `'1'` are considered equal.

**ANSI on:** behaves like pandas, `1 == '1'` is False.

Examples are as shown below:

```python
>>> pdf = pd.DataFrame({"int": [1, 2], "str": ["1", "2"]})
>>> psdf = ps.from_pandas(pdf)
>>> psdf["int"] == psdf["str"]
0    False
1    False
dtype: bool
>>> pdf["int"] == pdf["str"]
0    False
1    False
dtype: bool

# ANSI off
>>> spark.conf.set("spark.sql.ansi.enabled", False)
>>> psdf["int"] == psdf["str"]
0    True
1    True
dtype: bool
```

### Strict Casting
**ANSI off:** invalid casts (e.g., `'a' → int`) quietly became NULL.

**ANSI on:** the same casts raise errors.

Examples are as shown below:

```python
>>> pdf = pd.DataFrame({"str": ["a"]})
>>> psdf = ps.from_pandas(pdf)
>>> psdf["str"].astype(int)
Traceback (most recent call last):
...
pyspark.errors.exceptions.captured.NumberFormatException: [CAST_INVALID_INPUT] ...
>>> pdf["str"].astype(int)
Traceback (most recent call last):
...
ValueError: invalid literal for int() with base 10: 'a'

# ANSI off
>>> spark.conf.set("spark.sql.ansi.enabled", False)
>>> psdf["str"].astype(int)
0   NaN
Name: str, dtype: float64
```

### MultiIndex.to_series Return
**ANSI off:** returns each row as a list ([1, red]).

**ANSI on:** returns each row as a tuple ((1, red)), with the Runtime SQL Configuration `spark.sql.execution.pandas.structHandlingMode` set to `'row'`.

Examples are as shown below:

```python
>>> arrays = [[1,  2], ["red", "blue"]]
>>> pidx = pd.MultiIndex.from_arrays(arrays, names=("number", "color"))
>>> psidx = ps.from_pandas(pidx)

>>> spark.conf.set("spark.sql.execution.pandas.structHandlingMode", "row")
>>> psidx.to_series()
number  color
1       red       (1, red)
2       blue     (2, blue)
dtype: object
>>> pidx.to_series()
number  color
1       red       (1, red)
2       blue     (2, blue)
dtype: object

# ANSI off
>>> psidx.to_series()
number  color
1       red       [1, red]
2       blue     [2, blue]
dtype: object
```

## Related Configurations
1. **`compute.fail_on_ansi_mode` (Pandas API on Spark option)**
   - Controls whether Pandas API on Spark fails immediately when ANSI mode is enabled.
   - Now overridden by `compute.ansi_mode_support`.

2. **`compute.ansi_mode_support` (Pandas API on Spark option)**
   - Indicates whether ANSI mode is fully supported.

3. **`spark.sql.ansi.enabled` (Spark config)**
   - Native Spark setting that controls ANSI mode.